In [16]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Загрузим нужные (или не очень) библиотеки
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import optimize
import requests as requests
from bs4 import BeautifulSoup
import re
from sympy.solvers import solve
from sympy import Symbol
import urllib
import json
import pickle
import urllib.request
import os
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

In [ ]:
directory = (r'directory_name') #Вставьте путь директории вместо directory name

In [2]:
#запрос HTML кода, в нём пользователь вручную находит единственное значение (ctrl+F) 
#ссылку типа /cache/N72Hgd54/icharts/icharts.js  по значению icharts.js и задаёт её в параметр icharts (Ниже)
finam = ('https://www.finam.ru/profile/moex-akcii/gazprom/export/')
getJS = requests.get(finam)
print(getJS.text)

<!DOCTYPE html>
<html xmlns:og="http://ogp.me/ns#">
<head>
<title>Финам.ru - Экспорт котировок Акции ГАЗПРОМ ао - котировки, стоимость, цена акций, новости компании онлайн</title>
<meta name="Description" content="На finam.ru вы можете ознакомиться с котировками акции ГАЗПРОМ ао (GAZP) на рынке 'МосБиржа акции' в режиме реального времени - котировки онлайн, стоимость, графики, новости." />


<meta http-equiv="Content-Type" content="text/html; charset=windows-1251" />
<meta property="fb:app_id" content="481850738501742" />
<meta property="segmentsApiUrl" content="https://segments-api.finam.ru" />

<meta http-equiv="Pragma" content="no-cache" />
<meta http-equiv="Cache-Control" content="no-cache" />
<meta http-equiv="Expires" content="-1" />
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<link href="/favicon.ico?2" rel="shortcut icon" />

<!--eski.mobi-->
<script>!function(a,b,c,d,e){function g(a,c,d,e

In [18]:
#вставляет ссылку на icharts
icharts = 'https://www.finam.ru/cache/N72Hgd54/icharts/icharts.js'

In [19]:
#открывает, фильтрует данные из icharts, сохраняет icharts отдельно
urllib.request.urlretrieve(icharts, 'icharts.txt')
ttt = requests.get(icharts)
data = str(ttt.content)
data1 = data.replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('"', '').replace(':', ',').replace(' ', '')
#print(data1)

In [20]:
#сохраняет блок URL ссылок из icharts
z = open('aEmitentUrls.txt', 'w')
z.write(data1[data1.find("aEmitentUrls") + 1 : ])

777674

In [21]:
#открывает EmitentUrls и фильтрует только ссылки про облигации
df = pd.read_csv('aEmitentUrls.txt', sep = ',', header = None)
dfT = df.T
dfT.columns = ['URLs']
dfT_loc = dfT.loc[dfT['URLs'].str.contains("obligacii", na=False)]
dfT_loc.to_csv('URLS.csv')
urls = pd.read_csv('URLS.csv')
urls.columns = ['ne nyshno', 'URLs']
del urls ['ne nyshno']
urls

,URLs
0,gosobligacii-usa/10-yr-us-treasury
1,gosobligacii-usa/2-yr-us-treasury
2,gosobligacii-usa/30-yr-us-treasury
3,moex-obligacii/mfc-4finance-ltd-bo-01
4,gosobligacii-usa/5-yr-us-treasury
...,...
5165,moex-obligacii/yarosl-region-10
5166,moex-obligacii/yarosl-region-11
5167,moex-obligacii/yaroslavl-region-35016
5168,moex-obligacii/yaroslavl-region-35017_ru000a10...


In [22]:
pup = 'https://www.finam.ru/profile/' + (urls['URLs'][1:]) + '/export'

In [159]:
# Для примера работы программы можно создать массив из n случайных облигаций (в данном случае n = 5). Введите этот код, если хотите получить пример, подобный тому, что в руководстве
urls_example = urls.sample(5)
urls_example

,URLs
2186,moex-obligacii/lsr-001p-05_ru000a100zl8
472,moex-obligacii/alfa-bank-bo-17
1941,moex-obligacii/kobr-10
3859,moex-obligacii/szkc-04
3892,mosbirzha-vnespisochnye-obligacii/smarts-03


In [160]:
pup_example = 'https://www.finam.ru/profile/' + (urls_example['URLs'][1:]) + '/export'

In [161]:
#запуск Chrome
driver = webdriver.Chrome()
driver.get(pup[1])
#директорию скачивания txt файлов пользователь должен выбрать вручную в браузере до включения следующего блока (будет инструкция)

In [162]:
for j in pup:
    driver.get(j)
    time.sleep(4)
    elem = driver.find_element_by_id('issuer-profile-export-button')
    elem.click()
    time.sleep(5)
driver.close()

In [186]:
Prices = pd.DataFrame({"Date": [], "ISIN": [], "Open": [], "Close":[], "High":[], "Low": [], "Deals": [], "Volume": [], "Accrued interest": [], "Duration": [], "Yield to maturity": []})
Information = pd.DataFrame({"Bond": [],"ISIN-cod": [], "Номинальный обьём": [], "Размещённый обьём": [],"Начало размещ.": [], "Окончание размещ.": [],
                               "Дата погашения": [], "Номинал": [],"Рег. номер": [], "Дата рег": []})
                                   
# Создадим шаблон для выплат облигации
Payments = pd.DataFrame({"Date": [], "ISIN": [], "Coupon": [], "Coupon_Payment": []})



In [187]:
files = os.listdir(directory)
# Здесь происходит всё действо, собранное воедино (и по отдельности всё хорошо работает)
for i in range(len(pup)):
    file = files[i]
    egg = (directory + str("\\") + files[i])
    df1 = pd.read_csv(egg, sep = ",")
    if len(df1) == 0:
        New_Prices = pd.DataFrame({"Date": [], "Bond": [], "ISIN": [], "Open": [], "Close":[], "High":[], "Low": [], "Deals": [], "Volume": [], "Accrued interest": [], "Duration": [], "Yield to maturity": []})
        New_Payments = pd.DataFrame({"Date": [], "Bond": [], "ISIN": [], "Coupon": [], "Coupon_Payment": []})
        New_Information = pd.DataFrame({"Bond": [],"ISIN-cod": [], "Номинальный обьём": [], "Размещённый обьём": [],"Начало размещ.": [], "Окончание размещ.": [],
                               "Дата погашения": [], "Номинал": [],"Рег. номер": [], "Дата рег": []})        
        b = "Нет данных"
        New_Prices["Date"] = b  
        New_Prices["ISIN"] = b
        New_Prices["Open"] = b
        New_Prices["Close"] = b
        New_Prices["High"] = b
        New_Prices["Low"] = b
        New_Prices["Volume"] = b
        New_Prices["Deals"] = b
        New_Prices["Bond"] = file
        Prices = pd.concat([Prices, New_Prices], ignore_index=True)
        Payments = pd.concat([Payments, New_Payments], ignore_index=True)   
        Information = pd.concat([Information, New_Information], ignore_index=True)
    
    
    else:
        New_Prices = pd.DataFrame({"Date": [], "Bond": [], "ISIN": [], "Open": [], "Close":[], "High":[], "Low": [], "Deals": [], "Volume": [], "Accrued interest": [], "Duration": [], "Yield to maturity": []})
        New_Payments = pd.DataFrame({"Date": [], "Bond": [], "ISIN": [], "Coupon": [], "Coupon_Payment": []})
        New_Information = pd.DataFrame({"Bond": [],"ISIN-cod": [], "Номинальный обьём": [], "Размещённый обьём": [],"Начало размещ.": [], "Окончание размещ.": [],
                               "Дата погашения": [], "Номинал": [],"Рег. номер": [], "Дата рег": []})
        # Запишем название облигации и её код
        Bond = "ГПР"
        ISIN = df1["<TICKER>"][0]
        # Заполним шаблон скачанными данными
        New_Prices["Date"] = df1["<DATE>"]
        New_Prices["ISIN"] = df1["<TICKER>"]
        New_Prices["Open"] = df1["<OPEN>"]
        New_Prices["Close"] = df1["<CLOSE>"]
        New_Prices["High"] = df1["<HIGH>"]
        New_Prices["Low"] = df1["<LOW>"]
        New_Prices["Volume"] = df1["<VOL>"]
        New_Prices["Deals"] = New_Prices["Close"]/New_Prices["Volume"]
        New_Prices.Date = pd.to_datetime(New_Prices.Date, format="%Y%m%d")
        # Парсим Html-страницу с купонами
        a = ISIN
        # Достаём нужную ссылку через ISIN-код облигации
        url = "https://bonds.finam.ru/issue/search/default.asp?operatorIdName=&operatorTypeName=&quoteType=1&Code=&srchString=" + a + "&sectorId=&FieldId=0&status=0&currencyId=0&volumeFrom=&volumeTo=&couponRateFrom=&couponRateTo=&placementMethod=0&registrationDateFrom=&registrationDateTo=&regNumber=&amortization=0&operatorId=0&OperatorTypeId=0&placementFrom=&placementTo=&paymentFrom=&paymentTo=&couponDateFrom=&couponDateTo=&offerExecDateFrom=&offerExecDateTo=&YTMFrom=&YTMTo=&YTMOffer=on&isRPS=0&leaderDateFrom=&leaderDateTo=&liquidRange=0"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        soup.findAll('a')
        surl = soup.findAll('a', href = re.compile('details'))
        if len(surl) == 0:
            b = "Нет данных"
            New_Information["ISIN-cod"] = b
            New_Information["Номинальный обьём"] =  b
            New_Information["Размещённый обьём"] = b
            New_Information["Начало размещ."] = b
            New_Information["Окончание размещ."] = b
            New_Information["Дата погашения"] = b
            New_Information["Номинал"] = b
            New_Information["Рег. номер"] = b
            New_Information["Дата рег"] = b
            New_Payments.Date = b
            New_Payments.ISIN = b
            New_Payments.Coupon = b
            New_Payments.Coupon_Payment = b   
            Prices = pd.concat([Prices, New_Prices], ignore_index=True)
            Payments = pd.concat([Payments, New_Payments], ignore_index=True)
            Information = pd.concat([Information, New_Information], ignore_index=True)
        else:                  
            k = str(soup.findAll('a', href = re.compile('details'))[1]).replace('<a href="', '').replace('</a>', '').replace('<a class=', '').replace('<"selected-rich"', '').replace(' href=', '').replace('"', '').replace('selected-rich', '').replace('>', '').replace('Платежи', '')
            k = ('/'.join(k.split('/')[:3]))
            url1 = "https://bonds.finam.ru" + k + "00002/default.asp"
            response = requests.get(url1)
            soup = BeautifulSoup(response.text, 'lxml')
            # Заполнение df3 с разнообразной информацией об облигации
            contentBlock = soup.find('td', id="content-block")
            parametrs_em = contentBlock.findAll('td')
            parametrs_em_bond = contentBlock.findAll('h2')
            bond_string = str(parametrs_em_bond)
            bond_string2 = bond_string.split('>')[1]
            Bond = bond_string2.split('<')[0]
            nomValue = str(str(parametrs_em[3]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '')
            val = nomValue1[28:len(nomValue)]
            razmValue = str(str(parametrs_em[5]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '')
            Start = str(str(parametrs_em[6]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            End = str(str(parametrs_em[8]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            Pogashenie = str(str(parametrs_em[9]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            kNominal = str(parametrs_em[12]).replace('\xa0', ' ').replace('<td>Номинал: <span>', '').replace('</span>', '')
            Nominal = int((kNominal.split('<')[0]).replace(" ", '').replace('\r','').replace('RUR', ''))
            RegNumber = str(str(parametrs_em[15]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            Registration = str(str(parametrs_em[17]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            ISIN_cod = str(str(parametrs_em[18]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            Other_info = str(str(parametrs_em[30:]).replace('<span>', '').replace('</span>', '').replace('</td>', '').replace('<td>','')).replace('\xa0', '').replace('\r', '').replace('\t', '').replace('\n', '')
            New_Information.loc[0, "ISIN-cod"] = df1["<TICKER>"][1]
            New_Information.loc[0, "Номинальный обьём"] =  nomValue
            New_Information.loc[0, "Размещённый обьём"] = razmValue
            New_Information.loc[0, "Начало размещ."] = Start
            New_Information.loc[0, "Окончание размещ."] = End
            New_Information.loc[0, "Дата погашения"]= Pogashenie
            New_Information.loc[0, "Номинал"]= Nominal
            New_Information.loc[0, "Рег. номер"]= RegNumber
            New_Information.loc[0, "Дата рег"]= Registration
            New_Information.loc[0, "Bond"] = Bond
            New_Information
            t = soup.findAll('td', align ="center")
            s = soup.findAll('td', align ="right")
            # Находим количество платежей по облигации
            num_pay = int(len(t)/2)
            pay = range(num_pay)
            paym = pd.DataFrame({"pay": [0]}, index = pay)
            # Заполняем колонку с датами платежей
            for i in range(num_pay):
                paym.pay[i] = str(t[2*i+1])
              # Отфильтруем весь мусор из строки html страницы
            for i in range(len(paym)):
                paym.pay[i] = paym.pay[i].replace('<td align="center">', '').replace('#940018">', '').replace('<font color="', '').replace('</td>', '').replace(' ', '').replace('<fontcolor="#940018">', '').replace('</font>', '').replace('<font color', '').replace('<fontcolor', '')
            # Повторим тоже самое для купонных платежей    
            coup = pd.DataFrame({"coup1": [0]}, index = pay)
            for i in range(num_pay):
                coup.coup1[i] = str(s[5*i])
            for i in range(len(coup)):
                coup.coup1[i] = coup.coup1[i].replace('<td align="right">', '').replace('</td>', '').replace(' ', '')
            coup_pay = pd.DataFrame({"coup": [0]}, index = pay)
            for i in range(num_pay):
                coup_pay.coup[i] = str(s[5*i+2])
            coup_pay
            for i in range(len(coup_pay)):
                coup_pay.coup[i] = coup_pay.coup[i].replace('</td>', '').replace(' ', '').replace('<tdalign="right">', '')
            for i in range(len(coup_pay)):
                coup_pay.coup[i] = coup_pay.coup[i].replace(val, '')
            New_Payments.Date = paym.pay
            New_Payments.ISIN = df1['<TICKER>'][0]
            New_Payments.Coupon = coup.coup1
            New_Payments.Coupon_Payment = coup_pay.coup
            New_Payments.Bond = Bond
        # Данные сохраняются в очень странном формате, исправим это
            for i in range(len(New_Payments)):
                New_Payments.Date[i] = str(New_Payments.Date[i][0:10])    
            for i in range(len(New_Payments)):
                k = str(New_Payments.Coupon_Payment[i])
                k = ('\xa0'.join(k.split('\xa0')[:1]))
                New_Payments.Coupon_Payment[i] = k
            for i in range(len(New_Payments)):
                k = str(New_Payments.Coupon[i])
                k = ('\xa0'.join(k.split('\xa0')[:1]))
                New_Payments.Coupon[i] = k
            for i in range(len(coup_pay)):
                New_Payments.Coupon_Payment[i] = New_Payments.Coupon_Payment[i].replace(',' , '.')
            New_Payments.Coupon_Payment = pd.to_numeric(New_Payments.Coupon_Payment)
            nominal = New_Information['Номинал']
            New_Prices["Bond"] = Bond
            # Здесь заполнение трёх столбцов в Prices через функции
            
            # Соединим данные по новой облигации с уже имеющимися
            Prices = pd.concat([Prices, New_Prices], ignore_index=True)
            Payments = pd.concat([Payments, New_Payments], ignore_index=True)
            Information = pd.concat([Information, New_Information], ignore_index=True)

In [44]:
# Копируем сюда функции для расчёта

In [195]:
Information


,Bond,ISIN-cod,Номинальный обьём,Размещённый обьём,Начало размещ.,Окончание размещ.,Дата погашения,Номинал,Рег. номер,Дата рег
0,СМАРТС-03,RU000A0DDY44,Номинальный объем:1000000000RUR,,Дата начала размещения: 05.10.2004,Дата окончания размещения: ...,Дата погашения: 29.09.2009,1000.0,Рег. номер:4-03-00317-P,Дата рег:10.06.2004
1,СЗКК-04,RU000A0JRU46,Номинальный объем:5000000000RUR,Размещенный объем:5000000000RUR,Дата начала размещения: 21.10.2011,Дата окончания размещения: ...,Дата погашения: 26.09.2031,1000.0,Рег. номер:4-02-36388-R,Дата рег:01.09.2011Рег. орган:ФСФР
2,АЛЬФА-БАНК-БО-17,RU000A0JWUX9,Номинальный объем:5000000000RUR,Размещенный объем:5000000000RUR,Дата начала размещения: 04.10.2016,Дата окончания размещения: ...,Дата погашения: 16.09.2031,1000.0,Рег. номер:4B021701326B,Дата рег:06.02.2014Рег. орган:ММВБ


In [462]:
def ncd(dates, dates_of_payments, coupon_payments):
    # Функция берёт в качестве аргументов вектора Prices.Date, Payments.Date и Payments.Coupon_Payments, считает НКД для каждой даты
    # и выдаёт вектор Prices.Accrued_Interest, добавляющийся в таблицу вектор days_from_last_payment записывает количество дней,
    # прошедших со времени последнего платежа (длина вектора равна длине вектора Prices.Date)
    # calculation_days - вспомогательный массив. Мы для каждой даты выплаты расчитываем разницу в днях до текущей даты, проверяем, чтобы она была больше нуля (текущая дата после даты последней выплаты) и берём минимальное значение из оставшихся.
    # Таким образом, мы находим, сколько дней прошло с последнего платежа. Далее по формуле рассчитываем НКД для текущей даты
    # Базис - 365 дней в году
    k = len(dates)
    m = len(dates_of_payments)
    B = 365
    d1 = dates
    d2 = dates_of_payments
    # создаём вспомогательные вектора для расчётов
    calc1 = pd.DataFrame({"calculation_days": []})
    calc2 = pd.DataFrame({"days_from_last_payment": []})
    calc3 = pd.DataFrame({"Accrued_interest": []})
    if (m>0):
            for i in range(k):
                for j in range(m):
            # Считаем разницу между текущим днём (i) и всеми днями выплат (вектор Payments.Date)
            # Для прошедших платежей пишем 100000, они отсеются, когда мы будем искать ближайший будущий платёж 
                    calc1.calculation_days[j] = (d1[i]-d2[j]).days if (d1[i]-d2[j]).days > 0 else 100000 
          # Рассчитываем разницу между сегодняшним днём и ближайшим будущим платежом 
                p = min(calc1.calculation_days)
                if p == 100000:
                      calc2.days_from_last_payment[i] = 'There are no future payments'
                else:                                    
                      calc2.days_from_last_payment[i] = p  
                if (calc2.days_from_last_payment[i] == 'There are no future payments'):
                     calc3.Accrued_interest[i] = 'There are no future payments'
                else:
                         # Находим индекс нужного нам платежа и считаем НКД                                     
                    c = calc1.calculation_date.index[min(calc1.calculation_date)]
                    calc3.loc[i, "Accrued_interest"] = coupon_payments[c]*calc2.days_from_last_payment[i]/B  
            return(calc3.Accrued_interest)
    else:
        print('There are no payments')

In [430]:
def cf(date, dates_of_payments, coupon_payments, nominal):
    # Формируем датафрейм с предстоящими платежами по облигации (включая погашение номинала) и сроком до этих платежей на конкретный день
    cf1 = pd.DataFrame({"days": [],"payments": [], "years": []})
    m = len(dates_of_payments)
    for i in range(m):
    # Здесь мы перебираем все даты платежей и рассчитываем время до платежей в днях (и размер этих платежей)
        cf1.loc[i, 'days'] = (dates_of_payments[i]-date).days  
        cf1.loc[i, 'payments'] = coupon_payments[i]
    #Добавим погашение номинала
    cf1.loc[m, 'payments'] = nominal
    cf1.loc[m, 'days'] = cf1.days[m-1]
    #Удалим все строки с прошедшими платежами
    filter_days = (cf1.days > 0)    
    cf1 = cf1.loc[filter_days]
        # Добавим столбец со сроком в годах (пригодится для дальнейшего расчёта ставки)
    cf1.years = cf1.days/365
    
    return(cf1)

In [432]:
def dcf(date, dates_of_payments, coupon_payments, nominal, rate):
# Расчитывается дисконтированную сумму платежей по облигации при известной процентной ставке. Начисление процентов - дискретное

    cash_flow = cf(date, dates_of_payments, coupon_payments, nominal)
    k = len(cash_flow)
    dcf = pd.DataFrame({"dcf1": []})
    for i in range(k):
    # Дисконтируем каждый из платежей по ставке rate
        dcf.loc[i, "dcf1"] = cash_flow.payments[i]/(1+rate)**(cash_flow.years[i])
# Суммируем все получившиеся дисконтированные платежи    
    sum_dcf = sum(dcf.dcf1)
    return(sum_dcf)

In [470]:

def YTM(date, dates_of_payments, coupon_payments, nominal):
# Получим функцией cf таблицу с денежными потоками
# Получим цену закрытия облигации на нужную дату
    index1 = New_Prices[New_Prices.Date == date].index
# Решаем уравнение (цена облигации равна сумме её дисконтированных платежей) и находим ставку, по которой мы дисконтируем эти платежи (YTM)
    price = New_Prices.Close[index1]    # находим цену, соотвествующую текущей дате
    # Уравнение dcf(rate = x) - price = 0
    x = Symbol('x')
    ytm = solve(dcf(date, dates_of_payments, coupon_payments, nominal, rate = x) - price, x)
    return(ytm)

In [467]:
def duration(date, dates_of_payments, coupon_payments, nominal):
    cf1 = cf(date, dates_of_payments, coupon_payments, nominal)
    ytm = YTM(date, dates_of_payments, coupon_payments, nominal)
    index = New_Prices[New_Prices.Date == date].index
    if (len(index) == 0):
          return("error, dont have this date in DB")
    else:
        index1 = index[0]
        price = New_Prices.Close[index1]
        k = len(cf1)
        dcf = pd.DataFrame({"dcf1": []})
        for i in range(k):
            dcf.dcf1[i] = cf1.years[i]*(cf1.payments[i])/(1+ytm)**cf1.years[i]
        dur = sum(dcf.dcf1)/price
        return(dur)

In [192]:
Payments.to_csv("Payments")
Prices.to_csv("Prices")
Information.to_csv("Information")